In [9]:
import cv2
import numpy as np

In [10]:
def people_counter():
    global i, centroid
    for i, contour in enumerate(contours):
        # Find largest contour
        if 5000 < cv2.contourArea(contour):
            M = cv2.moments(contour)
            centroid = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

            # Check if previous centroid exists for this contour
            if len(prev_centroids) > i and prev_centroids[i] is not None:
                # Check if current centroid is to the left or right of previous centroid
                if centroid[0] > prev_centroids[i][0]:
                    direction = "Right"
                else:
                    direction = "Left"

                # Check if current centroid has crossed vertical line in the middle of the viewport
                if (prev_centroids[i][0] <= line_pos < centroid[0]) or (
                        prev_centroids[i][0] >= line_pos > centroid[0]):
                    if direction == "Right":
                        counts[i] += 1
                        break
                    else:
                        counts[i] -= 1
                        break

                # Draw contour and centroid on frame
                cv2.drawContours(frame, [contour], 0, (0, 255, 0), 2)
                cv2.circle(frame, centroid, 5, (0, 0, 255), -1)

            # Add new contour's centroid and direction to the lists
            else:
                prev_directions.append(None)
                prev_centroids.append(centroid)
                counts.append(0)

In [11]:
def frame_processing():
    global contours
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    # Threshold the frame to create a binary image
    _, thresh = cv2.threshold(gray, 700, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # Find contours in the binary image
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [12]:
def display_frame():
    # Draw vertical line in the middle of the viewport
    cv2.line(frame, (line_pos, 0), (line_pos, int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))), (0, 255, 255), 2)
    # Draw count for each contour on frame
    cv2.putText(frame, "Count: {}".format(sum(counts)), (10, 30 + 30 * i), font, font_scale, font_color, line_type)
    # Display frame
    cv2.imshow("Train Counter", frame)

In [13]:
def background_subtraction(frame):
    fgbg = cv2.createBackgroundSubtractorMOG2()
    fgmask = fgbg.apply(frame)
    return cv2.bitwise_and(frame, frame, mask=fgmask)

In [14]:
def __main__():
    global frame
    while True:
        # Read frame from video capture
        ret, frame = cap.read()
        # Check if frame was successfully read
        if not ret:
            print("Error reading frame from video stream")
            break
        # Apply background subtraction
        frame = background_subtraction(frame)
    
        
        # Convert frame to grayscale and apply Gaussian blur
        gray = frame_processing()
        # Process each contour
        people_counter()
        # Display frames
        display_frame()
        # Wait for key press and limit frame rate to 15 fps
        if cv2.waitKey(67) & 0xFF == ord('q'):
            break
        # Update previous centroid
        prev_centroid = centroid
    # Release video capture and destroy all windows
    cap.release()
    cv2.destroyAllWindows()

In [15]:
cv2.startWindowThread()

# Create video capture object
cap = cv2.VideoCapture("../test_footage.mp4")


# Define font settings for text overlay
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_color = (255, 255, 255)
line_type = 2

# Initialize variables
prev_directions = []
prev_centroids = []
counts = []
line_pos = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) / 2)
__main__()